In [1]:
from dask_jobqueue import PBSCluster

In [2]:
!qstat -u $USER


chadmin1: 
                                                            Req'd  Req'd   Elap
Job ID          Username Queue    Jobname    SessID NDS TSK Memory Time  S Time
--------------- -------- -------- ---------- ------ --- --- ------ ----- - -----
3427300.chadmin abanihi  economy  STDIN        5739   1   1    --  04:00 R 00:38


In [3]:
cluster = PBSCluster(
    cores=36, memory='100GB', walltime='03:30:00', queue='economy', project='NIOW0001'
)

In [4]:
cluster

In [ ]:
!qstat -u $USER

In [8]:
import xarray as xr
from dask.distributed import Client

In [7]:
client = Client(cluster)

In [11]:
dset = xr.open_mfdataset(
    "/glade/scratch/mclong/archive/g.e20.G1850ECOIAF.T62_g17.hot_bats_13c.002/ocn/hist/g.e20.G1850ECOIAF.T62_g17.hot_bats_13c.002.pop.h.????-??.nc",
    decode_times=False,
)

In [12]:
dset

<xarray.Dataset>
Dimensions:                                (d2: 2, lat_aux_grid: 395, moc_comp: 3, moc_z: 61, nlat: 384, nlon: 320, time: 60, transport_comp: 5, transport_reg: 2, z_t: 60, z_t_150m: 15, z_w: 60, z_w_bot: 60, z_w_top: 60)
Coordinates:
    transport_components                   (transport_comp) |S384 b'Total' ... b'Submeso Advection'
    transport_regions                      (transport_reg) |S384 b'Global Ocean - Marginal Seas' b'Atlantic Ocean + Mediterranean Sea + Labrador Sea + GIN Sea + Arctic Ocean + Hudson Bay'
  * z_t                                    (z_t) float32 500.0 ... 537500.0
  * z_t_150m                               (z_t_150m) float32 500.0 ... 14500.0
  * z_w                                    (z_w) float32 0.0 ... 525000.94
  * z_w_top                                (z_w_top) float32 0.0 ... 525000.94
  * z_w_bot                                (z_w_bot) float32 1000.0 ... 549999.06
  * lat_aux_grid                           (lat_aux_grid) float32 -79

In [93]:
drop_vars = [
    v
    for v in dset.variables
    if v not in ['NO3', 'PO4', 'time', 'ULONG', 'ULAT', 'time_bound', 'z_t']
]

In [94]:
len(dset.variables), len(drop_vars)

(499, 492)

In [151]:
sample_dset = dset.drop(drop_vars).isel(
    time=slice(0, 24), z_t=slice(0, 2), nlon=slice(0, 50), nlat=slice(0, 60)
)

In [152]:
sample_dset

<xarray.Dataset>
Dimensions:     (d2: 2, nlat: 60, nlon: 50, time: 24, z_t: 2)
Coordinates:
  * z_t         (z_t) float32 500.0 1500.0
    ULONG       (nlat, nlon) float64 321.1 322.3 323.4 ... 14.0 15.13 16.25
    ULAT        (nlat, nlon) float64 -78.95 -78.95 -78.95 ... -47.43 -47.43
  * time        (time) float64 396.0 424.0 455.0 ... 1.064e+03 1.095e+03
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    time_bound  (time, d2) float64 dask.array<shape=(24, 2), chunksize=(1, 2)>
    PO4         (time, z_t, nlat, nlon) float32 dask.array<shape=(24, 2, 60, 50), chunksize=(1, 2, 60, 50)>
    NO3         (time, z_t, nlat, nlon) float32 dask.array<shape=(24, 2, 60, 50), chunksize=(1, 2, 60, 50)>
Attributes:
    title:             g.e20.G1850ECOIAF.T62_g17.hot_bats_13c.002
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    contents:          Diagnostic and Prognostic Variables
    s

In [153]:
sample_dset.to_netcdf("/glade/work/abanihi/data/cesm_sample.nc")

In [154]:
!du -s  /glade/work/abanihi/data/ -h

4.3M	/glade/work/abanihi/data/


In [155]:
import zarr

compressor = zarr.Blosc(cname='zstd', clevel=3, shuffle=2)

In [157]:
sample_dset.to_zarr(
    '/glade/work/abanihi/data/cesm_sample/',
    encoding={'PO4': {'compressor': compressor}, 'NO3': {'compressor': compressor}},
    mode='w',
)

In [158]:
!du -s  /glade/work/abanihi/data/cesm_sample/ -h

798K	/glade/work/abanihi/data/cesm_sample/


In [159]:
xr.open_zarr('/glade/work/abanihi/data/cesm_sample/', decode_times=False)

<xarray.Dataset>
Dimensions:     (d2: 2, nlat: 60, nlon: 50, time: 24, z_t: 2)
Coordinates:
    ULAT        (nlat, nlon) float64 dask.array<shape=(60, 50), chunksize=(60, 50)>
    ULONG       (nlat, nlon) float64 dask.array<shape=(60, 50), chunksize=(60, 50)>
  * time        (time) float64 396.0 424.0 455.0 ... 1.064e+03 1.095e+03
  * z_t         (z_t) float32 500.0 1500.0
Dimensions without coordinates: d2, nlat, nlon
Data variables:
    NO3         (time, z_t, nlat, nlon) float32 dask.array<shape=(24, 2, 60, 50), chunksize=(1, 2, 60, 50)>
    PO4         (time, z_t, nlat, nlon) float32 dask.array<shape=(24, 2, 60, 50), chunksize=(1, 2, 60, 50)>
    time_bound  (time, d2) float64 dask.array<shape=(24, 2), chunksize=(1, 2)>
Attributes:
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    calendar:          All years have exactly  365 days.
    cell_methods:      cell_methods = time: mean ==> the variable values are ...
    contents:          Diagnostic a